<a href="https://colab.research.google.com/github/RendiZein/Metode-Rekayasa-Komputasi/blob/main/Introduction_to_LLMs_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Large Language Models (LLMs) Landscape

In [ ]:
# Import the function for loading Hugging Face pipelines
from transformers import pipeline

prompt = "The food was good, but service at the restaurant was a bit slow"

# Load the pipeline for sentiment classification
classifier = pipeline('text-classification', model=model_name)

# Pass the customer review to the model for prediction
prediction = classifier(prompt)
print(prediction)

In [ ]:
# Load the model pipeline for text summarization
summarizer = pipeline('summarization', model=model_name)

# Pass the long text to the model to summarize it
outputs = summarizer(long_text, max_length=50)

# Access and print the summarized text in the outputs variable
print(outputs[0]['summary_text'])

In [ ]:
# Load the model pipeline for question-answering
qa_model = pipeline("question-answering")
question = "For how long was the Eiffel Tower the tallest man-made structure in the world?"

# Pass the necessary inputs to the LLM pipeline for question-answering
outputs = qa_model(question=question, context=context)

# Access and print the answer
print(outputs['answer'])

In [ ]:
# Set transformer model hyperparameters
d_model = 512
n_heads = 8
num_encoder_layers = 6
num_decoder_layers = 6

# Create the transformer model and assign hyperparameters
model = nn.Transformer(
    d_model=d_model,
    nhead=n_heads,
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers
)

print(model)

# Building a Transformer Architecture

In [ ]:
# Subclass an appropriate PyTorch class
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_length):
        super(PositionalEncoder, self).__init__()
        self.d_model = d_model
        self.max_length = max_length

        # Initialize the positional encoding matrix
        pe = torch.zeros(max_length, d_model)

        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float) * -(math.log(10000.0) / d_model))

        # Calculate and assign position encodings to the matrix
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    # Update the embeddings tensor adding the positional encodings
    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

Why is positional encoding added to token embeddings?

a. To accelerate the training process

**b.To include sequence order information into the models**

c.To outperform RNN models

d.To remove long-range dependencies

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Set the number of attention heads
        self.num_heads = num_heads
        self.d_model = d_model
        self.head_dim = d_model // num_heads
		# Set up the linear transformations
        self.query_linear = nn.Linear(d_model,d_model)
        self.key_linear = nn.Linear(d_model,d_model)
        self.value_linear = nn.Linear(d_model,d_model)
        self.output_linear = nn.Linear(d_model,d_model)

In [ ]:
def split_heads(self, x, batch_size):
    # Split the sequence embeddings in x across the attention heads
    x = x.view(batch_size, -1, self.num_heads, self.head_dim)
    return x.permute(0, 2, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.head_dim)

In [ ]:
def compute_attention(self, query, key, mask=None):
    # Compute dot-product attention scores
    scores = torch.matmul(query, key.permute(1, 2, 0))
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float("-1e20"))
    # Normalize attention scores into attention weights
    attention_weights = F.softmax(scores, dim=-1)
    return attention_weights

In [ ]:
def forward(self, query, key, value, mask=None):
    batch_size = query.size(0)

    query = self.split_heads(self.query_linear(query), batch_size)
    key = self.split_heads(self.key_linear(key), batch_size)
    value = self.split_heads(self.value_linear(value), batch_size)

    attention_weights = self.compute_attention(query, key, mask)

    # Multiply attention weights by values, concatenate and linearly project outputs
    output = torch.matmul(attention_weights, value)
    output = output.view(batch_size, self.num_heads, -1, self.head_dim).permute(0, 2, 1, 3).contiguous().view(batch_size, -1, self.d_model)
    return self.output_linear(output)

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, d_ff, dropout, max_sequence_length):
        super(TransformerEncoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoder(d_model, max_sequence_length)
        # Define a stack of multiple encoder layers
        self.layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

    # Complete the forward pass method
    def forward(self, x, mask):
        x = self.embedding(x)
        x = self.positional_encoding(x)
        for layer in self.layers:
            x = layer(x, mask)
        return x

class ClassifierHead(nn.Module):
    def __init__(self, d_model, num_classes):
        super(ClassifierHead, self).__init__()
        # Add linear layer for multiple-class classification
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        logits = self.fc(x[:, 0, :])
        # Obtain log class probabilities upon raw outputs
        return F.log_softmax(logits, dim=-1)

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        # Initialize the causal (masked) self-attention and cross-attention
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForwardSubLayer(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, causal_mask, encoder_output, cross_mask):
        # Pass the necessary arguments to the causal self-attention and cross-attention
        self_attn_output = self.self_attn(x, x, x, causal_mask)
        x = self.norm1(x + self.dropout(self_attn_output))
        cross_attn_output = self.cross_attn(x, encoder_output, encoder_output, cross_mask)
        x = self.norm2(x + self.dropout(cross_attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [ ]:
# Initialize positional encoding layer and stack of EncoderLayer modules
class TransformerEncoder(nn.Module):

    def __init__(self, vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_len, dropout):
        super(TransformerEncoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_len)
        self.layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        x = self.embedding(x)
        x = self.positional_encoding(x)
        x = self.dropout(x)

        # Pass the sequence through each layer in the encoder
        for layer in self.layers:
            x = layer(x, mask)

        return x

class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_len, dropout):
        super(Transformer, self).__init__()
        # Initialize the encoder stack of the Transformer
        self.encoder = TransformerEncoder(vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_len, dropout)

    def forward(self, src, src_mask):
        encoder_output = self.encoder(src, src_mask)
        return encoder_output

# Harnessing Pre-trained LLMs

## Summarizing a product opinion

In [ ]:
print(f"Number of instances: {len(dataset['train'])}")

# Show the names of features in the training fold of the dataset
print(f"Feature names: {dataset['train'].column_names}")

# Encode the input example, obtain the summary, and decode it
example = dataset['train'][-2]['review_sents']
input_ids = tokenizer.encode("summarize: " + example, return_tensors="pt", max_length=512, truncation=True)
summary_ids = model.generate(input_ids, max_length=150)
summary = tokenizer.decode(
  summary_ids[0], skip_special_tokens=True)

print("\nOriginal Text (first 400 characters): \n", example[:400])
print("\nGenerated Summary: \n", summary)

## The Spanish phrasebook mission

In [ ]:
model_name = "Helsinki-NLP/opus-mt-en-es"

# Load the tokenizer and the model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

english_inputs = ["Hello", "Thank you", "How are you?", "Sorry", "Goodbye"]

# Encode the inputs, generate translations, decode, and print them
for english_input in english_inputs:
    input_ids = tokenizer.encode(english_input, return_tensors="pt")
    translated_ids = model.generate(input_ids)
    translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    print(f"English: {english_input} | Spanish: {translated_text}")

## Load and inspect a QA dataset

In [ ]:
# Load a specific subset of the dataset
mlqa = load_dataset("xtreme", name="MLQA.en.en")

question = mlqa["test"]["question"][0]
context = mlqa["test"]["context"][0]
print("Question: ", question)
print("Context: ", context)

# Initialize the tokenizer using the model checkpoint
tokenizer = AutoTokenizer.from_pretrained("deepset/minilm-uncased-squad2")

# Tokenize the inputs returning the result as tensors
inputs = tokenizer(question, context, return_tensors="pt")
print("First five encoded tokens: ", inputs["input_ids"][0][:5])

## Extract and decode the answer

In [ ]:
# Initialize the LLM upon the model checkpoint
model = AutoModelForQuestionAnswering.from_pretrained(model_ckp)

with torch.no_grad():
  # Forward-pass the input through the model
  outputs = model(**inputs)

# Get the most likely start and end answer position from the raw LLM outputs
start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits) + 1

# Access the tokenized inputs tensor to get the answer span
answer_span = inputs["input_ids"][0][start_idx:end_idx]

# Decode the answer span to get the extracted answer text
answer = tokenizer.decode(answer_span)
print("Answer: ", answer)

## Fine-tuning preparations

In [ ]:
# Load a pre-trained LLM, specifying its use for binary classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Set up training arguments with a batch size of 8 per GPU and 5 epochs
training_args = TrainingArguments(
    output_dir="./smaller_bert_finetuned",
    per_device_train_batch_size=8,
    num_train_epochs=5,
)
# Set up trainer, assigning previously set up training arguments
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

## The inside-out LLM

In [ ]:
# Initialize the trainer and assign a training and validation set to it
trainer = Trainer(model=model, args=training_args,
    			compute_metrics=compute_metrics,
    			train_dataset=emotions_encoded["train"],
    			eval_dataset=emotions_encoded["validation"],
    			tokenizer=tokenizer
)

# Training loop to fine-tune the model
# trainer.train()

input_texts = ["It's dark and rainy outside", "I love penguins!"]

# Tokenize the input sequences and pass them to the model
inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    outputs  = model(**inputs)

# Obtain class labels from raw predictions
predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()

for i, predicted_label in enumerate(predicted_labels):
    print(f"\n Input Text {i + 1}: {input_texts[i]}")
    print(f"Predicted Label: {predicted_label}")

# Evaluating and Leveraging LLMs in the Real World

## Calculating accuracy

In [ ]:
# Pass the four input texts (without labels) to the pipeline
predictions = sentiment_analysis([example["text"] for example in test_examples])

true_labels = [example["label"] for example in test_examples]
predicted_labels = [1 if pred["label"] == "POSITIVE" else 0 for pred in predictions]

# Load the accuracy metric
accuracy = evaluate.load("accuracy")

result = accuracy.compute(references=true_labels, predictions=predicted_labels)
print(result)

## Beyond accuracy: describing metrics

In [ ]:
# Load the accuracy, precision, recall and F1 score metrics
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

# Obtain a description of each metric
print(help(accuracy))
print(help(precision))
print(help(recall))
print(help(f1))

## Beyond accuracy: using metrics

In [ ]:
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

# Pass the examples to the pipeline, and obtain a list predicted labels
sentiment_analysis = pipeline("sentiment-analysis")
predictions = sentiment_analysis([example for example in test_examples])
predicted_labels = [1 if pred["label"] == "POSITIVE" else 0 for pred in predictions]

# Compute the metrics by comparing real and predicted labels
print(precision.compute(references=test_labels, predictions=predicted_labels))
print(recall.compute(references=test_labels, predictions=predicted_labels))
print(f1.compute(references=test_labels, predictions=predicted_labels))

## Perplexed about 2030

In [ ]:
# Encode the prompt, generate text and decode it
prompt_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(prompt_ids, max_length=20)
generated_text = tokenizer.decode(
  output[0], skip_special_tokens=True)

print("Generated Text: ", generated_text)

# Load and compute the perplexity score
perplexity = evaluate.load("perplexity", module_type="metric")
results = perplexity.compute(model_id='gpt2',
                             predictions=generated_text)
print("Perplexity: ", results['mean_perplexity'])

## A feast of LLM metrics

In [ ]:
# Load the rouge metric
rouge = evaluate.load("rouge")

predictions = ["""Pluto is a dwarf planet in our solar system, located in the Kuiper Belt beyond Neptune, and was formerly considered the ninth planet until its reclassification in 2006."""]
references = ["""Pluto is a dwarf planet in the solar system, located in the Kuiper Belt beyond Neptune, and was previously deemed as a planet until it was reclassified in 2006."""]

# Calculate the rouge scores between the predicted and reference summaries
results = rouge.compute(predictions=predictions,                     references=references)
print("ROUGE results: ", results)

In [ ]:
meteor = evaluate.load("meteor")

llm_outputs = ["He thought it right and necessary to become a knight-errant, roaming the world in armor, seeking adventures and practicing the deeds he had read about in chivalric tales."]
references = ["He believed it was proper and essential to transform into a knight-errant, traveling the world in armor, pursuing adventures, and enacting the heroic deeds he had encountered in tales of chivalry."]

# Compute and print the METEOR score
results = meteor.compute(predictions=llm_outputs, references=references)
print("Meteor: ", results)

In [ ]:
exact_match = evaluate.load("exact_match")

predictions = ["The cat sat on the mat.", "Theaters are great.", "It's like comparing oranges and apples."]
references = ["The cat sat on the mat?", "Theaters are great.", "It's like comparing apples and oranges."]

# Compute the exact match and print the results
results = exact_match.compute(predictions=predictions, references=references)
print("EM results: ", results)

## BLEU-proof translations

In [ ]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")

# Translate the first input sentence
translated_output = translator(input_sentence_1)

translated_sentence = translated_output[0]['translation_text']

print("Translated:", translated_sentence)

# Calculate BLEU metric for translation quality
results = bleu.compute(predictions=[translated_sentence], references=reference_1)
print(results)

In [ ]:
# Translate the input sentences, extract the translated text, and compute BLEU score
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")

translated_outputs = translator(input_sentences_2)

predictions = [translated_output['translation_text'] for translated_output in translated_outputs]

print(predictions)

results = bleu.compute(predictions=predictions, references=references_2)
print(results)